In [1]:
import json
import string
from itertools import islice
import pandas as pd

import numpy as np
from collections import OrderedDict
from datetime import datetime

In [2]:
date_decoding="%m/%d/%Y, %X"
date_encoding = "%Y-%m-%d"
dataPath = '../final-project/public/data/news_china.json'
with open(dataPath) as file:
    data = json.load(file)
    print(len(data))

1841


In [3]:
def rankedByDate(data):
    rankedDate = {}
    for news in data:
        date = datetime.strptime(news['time-stamp'], date_decoding).strftime(date_encoding)
        if date in rankedDate:
            if isUnique(rankedDate[date], news['headline']):
                rankedDate[date].append(news)
        else:
            rankedDate[date] = [news]
    return dict(OrderedDict(sorted(rankedDate.items(), key=lambda t: t[0])))

def isUnique(data, headline):
    for news in data:
        if headline == news['headline']:
            return False
    return True

def quantile(fq):
    indicator = np.quantile(list(fq.values()), .50)
    scope = list(filter(lambda x: x > indicator, list(fq.values())))
    return int(np.quantile(scope, .75)), int(np.quantile(scope, .50)), int(np.quantile(scope, .25))

def DictToList(fq):
    fq_list = []
    for key, value in fq.items():
        item = [key, value]
        fq_list.append(item)
    return fq_list

def ListToDict(List):
    Dict = {}
    for key, value in List:
        Dict[key] = value
    return Dict

In [4]:
ranked = rankedByDate(data)

In [5]:
with open('eastern_ranked.json', 'w') as file:
    json.dump(ranked, file)

In [6]:
stopWords = [
    'COVID',
    'covid',
    'Covid',
    'COVID-19',
    'covid-19',
    'Covid-19',
    'COVID19',
    'covid19',
    'Covid19'
    'SARS-CoV-2'
    'Coronavirus',
    'coronavirus',
    'virus',
    'wa',
    'ha',
    '’','”','“','—'
]

In [7]:
from keywordExtract import extractKeywords

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/yhkuo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/yhkuo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/yhkuo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/yhkuo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
df = {}
for day, news in ranked.items():
    print(day)
    articles = []
    for each in news:
        articles.append(each['content'])
    extract = extractKeywords()
    extract.analyze(articles, window_size =4, lower=False, stopwords=stopWords)
    result = extract.get_keywords(100)
    amplified = []
    for key, value in result:
        amplified.append([key, int(value*10)])
    a_dict = ListToDict(amplified)
    final = {}
    final['wordcloud'] = amplified
    final['wordcloud_scope'] = {}
    final['wordcloud_scope']['Q3'], final['wordcloud_scope']['Q2'], final['wordcloud_scope']['Q1'] = quantile(a_dict)
    df[day] = final

2020-01-15
2020-01-19
2020-01-20
2020-01-21
2020-01-22
2020-01-23
2020-01-24
2020-01-25
2020-01-26
2020-01-27
2020-01-28
2020-01-29
2020-01-30
2020-01-31
2020-02-01
2020-02-02
2020-02-03
2020-02-04
2020-02-05
2020-02-06
2020-02-07
2020-02-08
2020-02-09
2020-02-10
2020-02-11
2020-02-12
2020-02-13
2020-02-14
2020-02-15
2020-02-16
2020-02-17
2020-02-18
2020-02-19
2020-02-20
2020-02-21
2020-02-22
2020-02-23
2020-02-24
2020-02-25
2020-02-26
2020-02-27
2020-02-28
2020-02-29
2020-03-01
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-07
2020-03-08
2020-03-09
2020-03-10
2020-03-11
2020-03-12


In [9]:
with open('../final-project/public/data/eastern_wordcloud.json', 'w') as file:
    json.dump(df, file)